<a href="https://colab.research.google.com/github/WhiteTiger-y/BuildForBharat-Ai-Techies/blob/main/Image%20to%20Text%20improved%20Results%20with%20LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers Pillow requests
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [2]:
from IPython import get_ipython
from IPython.display import display
from PIL import Image
import requests
import cv2
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from huggingface_hub import snapshot_download
from pathlib import Path

In [3]:
# Load the image captioning model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [4]:
torch.random.manual_seed(0)
llm_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
llm_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [10]:
pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
)

In [11]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [12]:
def process_video(video_path):
    """
    Extract 1 frame per second from the video, regardless of the video's frame rate.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get frames per second of the video
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Extract 1 frame per second
        if count % fps == 0:  # Capture only every fps-th frame (1 frame per second)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(frame_rgb))
        count += 1
    cap.release()
    return frames

In [15]:
video_path = "/content/asmr_street_respect_shortsfeed_shortsvideo_shortsyoutube_summer.mp4"  # Replace with your video path
frames = process_video(video_path)

In [16]:

for i, frame in enumerate(frames):
    inputs = processor(frame, return_tensors="pt")
    out = model.generate(**inputs)
    initial_caption = processor.decode(out[0], skip_special_tokens=True)
    print(f"Frame {i + 1} - Initial Caption: {initial_caption}")

    # Refine the caption using the Phi-3.5-mini-instruct model
    messages = [
        {"role": "system", "content": "You are guiding a blind person. Describe the scene with alerts: objects, actions, sounds."},
        {"role": "user", "content": f"Describe this for a blind person: {initial_caption}"},
    ]
    refined_caption = pipe(messages, **generation_args)[0]['generated_text']
    print(f"Frame {i + 1} - Refined Caption: {refined_caption}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Frame 1 - Initial Caption: cars are parked on the side of the road near a red painted curb


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Frame 1 - Refined Caption:  I'll describe the scene in detail, focusing on the sensory aspects that you can perceive:

Imagine you are standing on the sidewalk, feeling the cool pavement beneath your feet. The air around you is filled with the gentle hum of traffic in the distance. As you move forward, you can feel the subtle vibrations of the road caused by the passing cars.

As you continue to walk, you come across a line of parked cars. They are neatly arranged along the curb, which is painted a vivid red color. The red paint creates a stark contrast against the gray asphalt, making it easier for you to identify the boundary between the road and the parking area.

The cars themselves are mostly sedans and small SUVs, their doors closed and windows rolled up. You can hear the faint sound of the engines idling, a low, steady rhythm that fades as you get closer. The smell of recently used gasoline and rubber from the tires mingles with the fresh air, creating a unique scent that is fam